In [351]:
from sklearn.cluster import KMeans
import numpy as np
import csv
import math
import matplotlib.pyplot
import random
from matplotlib import pyplot as plt

In [352]:
maxAcc = 0.0
maxIter = 0
C_Lambda = 1
TrainingPercent = 80
ValidationPercent = 10
TestPercent = 10
PHI = []
IsSynthetic = False

In [353]:
def CreateDataMatrixSamePair(filePath):
    target = []
    count= 0
    addMatrix =[]
    subMatrix = []
    with open(filePath, 'rU') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            target.append(int(row[2]))
            data = []
            data1= []
            with open('HumanObserved-Features-Data.csv', 'rU') as f1:
                reader1 = csv.reader(f1) 
                next(reader1)
                for row1 in reader1:
                    if(row1[1] == row[0]):
                        for column in row1:
                            data.append(column)
                    if(row1[1] == row[1]):
                        for column in row1:
                            data1.append(column)
                        break
                data=data[2:]
                data=[int(data) for data in data]
                data1=data1[2:]
                data1=[int(data1) for data1 in data1]
                add = data + data1
                sub = [abs((data - data1))  for data,data1 in zip(data,data1)]
                addMatrix.append(add)
                subMatrix.append(sub)
    with open("diffn_pairs.csv", 'rU') as f:
        reader = csv.reader(f)
        next(reader)
        for row in reader:
            if(count) < 1300:
                target.append(int(row[2]))
                data = []
                data1= []
                with open('HumanObserved-Features-Data.csv', 'rU') as f1:
                    reader1 = csv.reader(f1) 
                    next(reader1)
                    for row1 in reader1:
                        if(row1[1] == row[0]):
                            for column in row1:
                                data.append(column)
                        if(row1[1] == row[1]):
                            for column in row1:
                                data1.append(column)
                            break
                    data=data[2:]
                    data=[int(data) for data in data]
                    data1=data1[2:]
                    data1=[int(data1) for data1 in data1]
                    add = data + data1
                    sub = [abs((data - data1))  for data,data1 in zip(data,data1)]
                    addMatrix.append(add)
                    subMatrix.append(sub)
            count = count +1
    return addMatrix , subMatrix, target

def GenerateTrainingTarget(rawTraining,TrainingPercent = 80):
    TrainingLen = int(math.ceil(len(rawTraining)*(TrainingPercent*0.01)))
    t           = rawTraining[:TrainingLen]
    #print(str(TrainingPercent) + "% Training Target Generated..")
    return t

def GenerateTrainingDataMatrix(rawData, TrainingPercent = 80):
    T_len = int(math.ceil(len(rawData[0])*0.01*TrainingPercent))
    d2 = rawData[:,0:T_len]
    #print(str(TrainingPercent) + "% Training Data Generated..")
    return d2

def GenerateValData(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData[0])*ValPercent*0.01))
    V_End = TrainingCount + valSize
    dataMatrix = rawData[:,TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Data Generated..")  
    return dataMatrix

def GenerateValTargetVector(rawData, ValPercent, TrainingCount): 
    valSize = int(math.ceil(len(rawData)*ValPercent*0.01))
    V_End = TrainingCount + valSize
    t =rawData[TrainingCount+1:V_End]
    #print (str(ValPercent) + "% Val Target Data Generated..")
    return t

def GenerateBigSigma(Data, MuMatrix,TrainingPercent,IsSynthetic):
    BigSigma    = np.zeros((len(Data),len(Data)))
    DataT       = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))        
    varVect     = []
    for i in range(0,len(DataT[0])):
        vct = []
        for j in range(0,int(TrainingLen)):
            vct.append(Data[i][j])    
        varVect.append(np.var(vct))
    
    for j in range(len(Data)):
        BigSigma[j][j] = varVect[j]
    if IsSynthetic == True:
        BigSigma = np.dot(3,BigSigma)
    else:
        BigSigma = np.dot(200,BigSigma)
    ##print ("BigSigma Generated..")
    return BigSigma


def GetScalar(DataRow,MuRow, BigSigInv):  
    R = np.subtract(DataRow,MuRow)
    T = np.dot(BigSigInv,np.transpose(R))  
    L = np.dot(R,T)
    return L

def GetRadialBasisOut(DataRow,MuRow, BigSigInv):    
    phi_x = math.exp(-0.5*GetScalar(DataRow,MuRow,BigSigInv))
    return phi_x

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

def GetWeightsClosedForm(PHI, T, Lambda):
    Lambda_I = np.identity(len(PHI[0]))
    for i in range(0,len(PHI[0])):
        Lambda_I[i][i] = Lambda
    PHI_T       = np.transpose(PHI)
    PHI_SQR     = np.dot(PHI_T,PHI)
    PHI_SQR_LI  = np.add(Lambda_I,PHI_SQR)
    PHI_SQR_INV = np.linalg.inv(PHI_SQR_LI)
    INTER       = np.dot(PHI_SQR_INV, PHI_T)
    W           = np.dot(INTER, T)
    ##print ("Training Weights Generated..")
    return W

def GetPhiMatrix(Data, MuMatrix, BigSigma, TrainingPercent = 80):
    DataT = np.transpose(Data)
    TrainingLen = math.ceil(len(DataT)*(TrainingPercent*0.01))         
    PHI = np.zeros((int(TrainingLen),len(MuMatrix))) 
    BigSigInv = np.linalg.inv(BigSigma)
    for  C in range(0,len(MuMatrix)):
        for R in range(0,int(TrainingLen)):
            PHI[R][C] = GetRadialBasisOut(DataT[R], MuMatrix[C], BigSigInv)
    #print ("PHI Generated..")
    return PHI

def GetValTest(VAL_PHI,W):
    Y = np.dot(W,np.transpose(VAL_PHI))
    ##print ("Test Out Generated..")
    return Y

def GetErms(VAL_TEST_OUT,ValDataAct):
    sum = 0.0
    t=0
    accuracy = 0.0
    counter = 0
    val = 0.0
    for i in range (0,len(VAL_TEST_OUT)):
        sum = sum + math.pow((ValDataAct[i] - VAL_TEST_OUT[i]),2)
        if(int(np.around(VAL_TEST_OUT[i], 0)) == ValDataAct[i]):
            counter+=1
    accuracy = (float((counter*100))/float(len(VAL_TEST_OUT)))
    ##print ("Accuracy Generated..")
    ##print ("Validation E_RMS : " + str(math.sqrt(sum/len(VAL_TEST_OUT))))
    return (str(accuracy) + ',' +  str(math.sqrt(sum/len(VAL_TEST_OUT))))

In [354]:
Add, Subtract, Target =  CreateDataMatrixSamePair('same_pairs.csv')
shuffledDataMatrix = list(zip(Add, Subtract, Target))
random.shuffle(shuffledDataMatrix)
Add, Subtract, Target = zip(*shuffledDataMatrix)
Add = np.transpose(Add)
Subtract = np.transpose(Subtract)
print(np.array(Target).shape)
print(np.array(Add).shape)
print(np.array(Subtract).shape)
#print(Subtract)

D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:6: DeprecationWarning: 'U' mode is deprecated
  
D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:13: DeprecationWarning: 'U' mode is deprecated
  del sys.path[0]
D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:32: DeprecationWarning: 'U' mode is deprecated
D:\MS\1stSem\Anaconda\lib\site-packages\ipykernel_launcher.py:40: DeprecationWarning: 'U' mode is deprecated


(2091,)
(18, 2091)
(9, 2091)


## Prepare Training Data

In [355]:
TrainingTarget = np.array(GenerateTrainingTarget(Target,TrainingPercent))
TrainingAddData   = GenerateTrainingDataMatrix(Add,TrainingPercent)
TrainingSubtractData   = GenerateTrainingDataMatrix(Subtract,TrainingPercent)
print(TrainingTarget.shape)
print(TrainingAddData.shape)
print(TrainingSubtractData.shape)

(1673,)
(18, 1673)
(9, 1673)


## Prepare Validation Data

In [356]:
ValDataAct = np.array(GenerateValTargetVector(Target,ValidationPercent, (len(TrainingTarget))))
ValDataAdd    = GenerateValData(Add,ValidationPercent, (len(TrainingTarget)))
ValDataSubtract    = GenerateValData(Subtract,ValidationPercent, (len(TrainingTarget)))
print(ValDataAct)
print(ValDataAdd.shape)
print(ValDataSubtract)

[1 1 1 0 1 0 1 0 1 0 0 0 0 0 1 1 1 1 1 0 1 0 1 1 0 0 0 1 1 1 0 1 0 1 0 1 0
 1 0 1 1 0 0 0 0 1 1 0 0 1 1 0 1 0 1 0 0 1 0 1 0 1 1 1 1 0 0 1 0 0 0 0 1 1
 1 0 0 1 1 0 0 1 1 0 1 0 0 1 0 1 0 1 0 0 0 1 1 0 0 1 0 0 0 0 1 0 0 1 0 0 0
 0 0 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 1 1 1 0 1 1 0 0 1 0 0 1 1 0 1 0 0 0 1 0
 0 0 1 0 0 1 1 0 0 0 1 0 1 0 1 0 1 0 0 1 0 1 0 0 0 1 1 0 0 0 0 0 0 0 0 1 0
 0 0 1 0 0 0 0 0 0 0 0 0 0 1 0 0 1 1 0 0 0 0 0 0]
(18, 209)
[[1 2 0 ... 0 1 1]
 [0 0 2 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [2 0 0 ... 0 0 0]
 [0 4 3 ... 3 1 1]
 [0 0 1 ... 0 0 0]]


## Prepare Test Data

In [357]:
TestDataAct = np.array(GenerateValTargetVector(Target,TestPercent, (len(TrainingTarget)+len(ValDataAct))))
TestDataAdd = GenerateValData(Add,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
TestDataSubtract= GenerateValData(Subtract,TestPercent, (len(TrainingTarget)+len(ValDataAct)))
print(TestDataAct.shape)
print(TestDataAdd.shape)
print(TestDataSubtract.shape)

(208,)
(18, 208)
(9, 208)


## Closed Form Solution For Feature Addition

In [358]:
ErmsArr = []
AccuracyArr = []
M= 18
kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingAddData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(Add, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrix(Add, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrix(TestDataAdd, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrix(ValDataAdd, Mu, BigSigma, 100)

In [359]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

(18, 18)
(18, 18)
(1673, 18)
(18,)
(209, 18)
(208, 18)


## Finding Erms on training, validation and test set

In [360]:
TR_TEST_OUT  = GetValTest(TRAINING_PHI,W)
VAL_TEST_OUT = GetValTest(VAL_PHI,W)
TEST_OUT     = GetValTest(TEST_PHI,W)

TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))
ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))
TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))

In [361]:
print ("-------Closed Form with Radial Basis Function For Feature Addition-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))
print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))
print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))
print ("Accuracy Training   = " + str(float(TrainingAccuracy.split(',')[0])))
print ("Accuracy Validation = " + str(float(ValidationAccuracy.split(',')[0])))
print ("Accuracy Testing    = " + str(float(TestAccuracy.split(',')[0])))

-------Closed Form with Radial Basis Function For Feature Addition-------
----------------------------------------------------
E_rms Training   = 0.41505497229888205
E_rms Validation = 0.420168114210727
E_rms Testing    = 0.41385602670330085
Accuracy Training   = 78.00358637178721
Accuracy Validation = 77.51196172248804
Accuracy Testing    = 79.8076923076923


## Gradient Descent solution for Linear Regression For Feature Addition

In [362]:
W_Now        = np.dot(220, W)
La           = 2
learningRate = 0.02
L_Erms_Test  = []
W_Mat        = []

for i in range(0,1601):
    
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next
    
    #-----------------TrainingData Accuracy---------------------#
TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    
    #-----------------ValidationData Accuracy---------------------#
VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    
    #-----------------TestingData Accuracy---------------------#
TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
Erms_Test = GetErms(TEST_OUT,TestDataAct)

In [363]:
print ("-------Gradient Descent For Feature Addition-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(Erms_TR.split(',')[1])))
print ("E_rms Validation = " + str(float(Erms_Val.split(',')[1])))
print ("E_rms Testing    = " + str(float(Erms_Test.split(',')[1])))
print ("Accuracy Training   = " + str(float(Erms_TR.split(',')[0])))
print ("Accuracy Validation = " + str(float(Erms_Val.split(',')[0])))
print ("Accuracy Testing    = " + str(float(Erms_Test.split(',')[0])))

-------Gradient Descent For Feature Addition-------
----------------------------------------------------
E_rms Training   = 0.5183955105338308
E_rms Validation = 0.5260165945256882
E_rms Testing    = 0.504649241377842
Accuracy Training   = 62.10400478182905
Accuracy Validation = 60.76555023923445
Accuracy Testing    = 64.42307692307692


## Closed Form Solution For Feature Subtraction

In [364]:
M= 9

kmeans = KMeans(n_clusters=M, random_state=0).fit(np.transpose(TrainingSubtractData))
Mu = kmeans.cluster_centers_

BigSigma     = GenerateBigSigma(Subtract, Mu, TrainingPercent,IsSynthetic)
TRAINING_PHI = GetPhiMatrix(Subtract, Mu, BigSigma, TrainingPercent)
W            = GetWeightsClosedForm(TRAINING_PHI,TrainingTarget,(C_Lambda)) 
TEST_PHI     = GetPhiMatrix(TestDataSubtract, Mu, BigSigma, 100) 
VAL_PHI      = GetPhiMatrix(ValDataSubtract, Mu, BigSigma, 100)

In [365]:
print(Mu.shape)
print(BigSigma.shape)
print(TRAINING_PHI.shape)
print(W.shape)
print(VAL_PHI.shape)
print(TEST_PHI.shape)

(9, 9)
(9, 9)
(1673, 9)
(9,)
(209, 9)
(208, 9)


## Finding Erms on training, validation and test set

In [366]:
TR_TEST_OUT  = GetValTest(TRAINING_PHI,W)
VAL_TEST_OUT = GetValTest(VAL_PHI,W)
TEST_OUT     = GetValTest(TEST_PHI,W)

TrainingAccuracy   = str(GetErms(TR_TEST_OUT,TrainingTarget))
ValidationAccuracy = str(GetErms(VAL_TEST_OUT,ValDataAct))
TestAccuracy       = str(GetErms(TEST_OUT,TestDataAct))

In [367]:
print ("-------Closed Form with Radial Basis Function For Feature Subtraction-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(TrainingAccuracy.split(',')[1])))
print ("E_rms Validation = " + str(float(ValidationAccuracy.split(',')[1])))
print ("E_rms Testing    = " + str(float(TestAccuracy.split(',')[1])))
print ("Accuracy Training   = " + str(float(TrainingAccuracy.split(',')[0])))
print ("Accuracy Validation = " + str(float(ValidationAccuracy.split(',')[0])))
print ("Accuracy Testing    = " + str(float(TestAccuracy.split(',')[0])))

-------Closed Form with Radial Basis Function For Feature Subtraction-------
----------------------------------------------------
E_rms Training   = 0.44508918059237706
E_rms Validation = 0.4456546821695326
E_rms Testing    = 0.4412212774339133
Accuracy Training   = 71.1297071129707
Accuracy Validation = 69.377990430622
Accuracy Testing    = 73.07692307692308


##  Gradient Descent solution for Linear Regression For Feature Subtraction

In [368]:
W_Now        = np.dot(220, W)
La           = 0.5
learningRate = 0.02
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for i in range(0,1270):
    
    #print ('---------Iteration: ' + str(i) + '--------------')
    Delta_E_D     = -np.dot((TrainingTarget[i] - np.dot(np.transpose(W_Now),TRAINING_PHI[i])),TRAINING_PHI[i])
    La_Delta_E_W  = np.dot(La,W_Now)
    Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
    Delta_W       = -np.dot(learningRate,Delta_E)
    W_T_Next      = W_Now + Delta_W
    W_Now         = W_T_Next
    
    #-----------------TrainingData Accuracy---------------------#
    TR_TEST_OUT   = GetValTest(TRAINING_PHI,W_T_Next) 
    Erms_TR       = GetErms(TR_TEST_OUT,TrainingTarget)
    L_Erms_TR.append(float(Erms_TR.split(',')[1]))
    
    #-----------------ValidationData Accuracy---------------------#
    VAL_TEST_OUT  = GetValTest(VAL_PHI,W_T_Next) 
    Erms_Val      = GetErms(VAL_TEST_OUT,ValDataAct)
    L_Erms_Val.append(float(Erms_Val.split(',')[1]))
    
    #-----------------TestingData Accuracy---------------------#
    TEST_OUT      = GetValTest(TEST_PHI,W_T_Next) 
    Erms_Test = GetErms(TEST_OUT,TestDataAct)
    L_Erms_Test.append(float(Erms_Test.split(',')[1]))

In [369]:
print ("-------Gradient Descent For Feature Subtraction-------")
print ('----------------------------------------------------')
print ("E_rms Training   = " + str(float(Erms_TR.split(',')[1])))
print ("E_rms Validation = " + str(float(Erms_Val.split(',')[1])))
print ("E_rms Testing    = " + str(float(Erms_Test.split(',')[1])))
print ("Accuracy Training   = " + str(float(Erms_TR.split(',')[0])))
print ("Accuracy Validation = " + str(float(Erms_Val.split(',')[0])))
print ("Accuracy Testing    = " + str(float(Erms_Test.split(',')[0])))

-------Gradient Descent For Feature Subtraction-------
----------------------------------------------------
E_rms Training   = 0.5585961360230975
E_rms Validation = 0.5551489231934809
E_rms Testing    = 0.56444919080074
Accuracy Training   = 37.89599521817095
Accuracy Validation = 39.23444976076555
Accuracy Testing    = 35.57692307692308


In [370]:
def sigmoid(scores):
    return 1 / (1 + np.exp(-scores))

## Logistic Regression solution for Feature Subtraction

In [371]:
W_Now        = np.zeros(9)
La           = 2
learningRate = 0.05
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []

for j in range(0,20):
    for i in range(0,1600):
        scores = np.dot(W_Now, TrainingSubtractData)
        prediction  = sigmoid(scores)
        #print ('---------Iteration: ' + str(i) + '--------------')
        error =  prediction - TrainingTarget
        Delta_E_D     = np.dot((TrainingSubtractData), error)/TrainingTarget.size
        #La_Delta_E_W  = np.dot(La,W_Now)
        #Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
        Delta_W       = -np.dot(learningRate,Delta_E_D)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next

    #-----------------Training Data Accuracy---------------------#
right=0
wrong=0
TR_TEST=[]
TR_TEST_OUT   =  sigmoid(np.dot(W_Now, TrainingSubtractData)) 
for res in TR_TEST_OUT:
    if (res>0.5):
        TR_TEST.append(1)
    else:
        TR_TEST.append(0)
for i,j in zip(TR_TEST, TrainingTarget):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Training Accuracy: " + str(right/(right+wrong)*100))
     
    #-----------------Validation Data Accuracy---------------------# 
right=0
wrong=0
VAL_TEST=[] 
VAL_TEST_OUT  = sigmoid(np.dot(W_Now, ValDataSubtract))
for res in VAL_TEST_OUT:
    if (res>0.5):
        VAL_TEST.append(1)
    else:
        VAL_TEST.append(0)
for i,j in zip(VAL_TEST, ValDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Validation Accuracy: " + str(right/(right+wrong)*100))
    
    #-----------------Testing Data Accuracy---------------------#
right=0
wrong=0
TEST =[]
TEST_OUT =  sigmoid(np.dot(W_Now, TestDataSubtract))  
for res in TEST_OUT:
    if (res>0.5):
        TEST.append(1)
    else:
        TEST.append(0)
for i,j in zip(TEST, TestDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Testing Accuracy: " + str(right/(right+wrong)*100))

Training Accuracy: 69.27674835624626
Validation Accuracy: 72.72727272727273
Testing Accuracy: 70.67307692307693


## Logistic Regression solution for Feature Addition

In [372]:
W_Now        = np.zeros(18)
La           = 2
learningRate = 0.05
L_Erms_Val   = []
L_Erms_TR    = []
L_Erms_Test  = []
W_Mat        = []
for j in range(0,20):
    for i in range(0,1600):

        scores = np.dot(W_Now, TrainingAddData)
        prediction  = sigmoid(scores)
        #print ('---------Iteration: ' + str(i) + '--------------')
        error =  prediction - TrainingTarget
        Delta_E_D     = np.dot(TrainingAddData, error)
        #La_Delta_E_W  = np.dot(La,W_Now)
        #Delta_E       = np.add(Delta_E_D,La_Delta_E_W)    
        Delta_W       = -np.dot(learningRate,Delta_E_D)
        W_T_Next      = W_Now + Delta_W
        W_Now         = W_T_Next

#-----------------Training Data Accuracy---------------------#
right=0
wrong=0
TR_TEST=[]
TR_TEST_OUT   =  sigmoid(np.dot(W_Now, TrainingAddData)) 
for res in TR_TEST_OUT:
    if (res>0.5):
        TR_TEST.append(1)
    else:
        TR_TEST.append(0)
for i,j in zip(TR_TEST, TrainingTarget):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Training Accuracy: " + str(right/(right+wrong)*100))
    
    #-----------------ValidationData Accuracy---------------------# 
right=0
wrong=0
VAL_TEST=[] 
VAL_TEST_OUT  = sigmoid(np.dot(W_Now, ValDataAdd))
for res in VAL_TEST_OUT:
    if (res>0.5):
        VAL_TEST.append(1)
    else:
        VAL_TEST.append(0)
for i,j in zip(VAL_TEST, ValDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Validation Accuracy: " + str(right/(right+wrong)*100))
    
    #-----------------TestingData Accuracy---------------------#
right=0
wrong=0
TEST =[]
TEST_OUT =  sigmoid(np.dot(W_Now, TestDataAdd))  
for res in TEST_OUT:
    if (res>0.5):
        TEST.append(1)
    else:
        TEST.append(0)
for i,j in zip(TEST, TestDataAct):
    if i==j:
        right = right +1
    else:
        wrong = wrong +1
print("Testing Accuracy: " + str(right/(right+wrong)*100))

Training Accuracy: 90.9145248057382
Validation Accuracy: 88.03827751196172
Testing Accuracy: 92.78846153846155


## Neural Network Solution for Feature Subtraction

In [373]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras.utils import np_utils
import numpy as np

TrainingTarget1 = np_utils.to_categorical(np.array(TrainingTarget),2)

In [374]:
input_size = 9
drop_out = 0.2
first_dense_layer_nodes  =256
second_dense_layer_nodes = 2

def get_model():
    model = Sequential()
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(first_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.summary()
    
    model.compile(optimizer='Adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [375]:
model = get_model()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_77 (Dense)             (None, 256)               2560      
_________________________________________________________________
activation_77 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_51 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_78 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_78 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_52 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_79 (Dense)             (None, 2)                 514       
__________

In [376]:
validation_data_split = 0
num_epochs = 1000
model_batch_size = 1000
tb_batch_size = 32
early_patience = 100
tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')
history = model.fit(np.transpose(TrainingSubtractData)
                    , TrainingTarget1
                    , validation_split=validation_data_split
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb])


Epoch 1/1000
1673/1673 [==============================] - 1s 493us/step - loss: 0.6936 - acc: 0.5152
Epoch 2/1000
1673/1673 [==============================] - 0s 58us/step - loss: 0.6689 - acc: 0.6192
Epoch 3/1000
1000/1673 [================>.............] - ETA: 0s - loss: 0.6742 - acc: 0.6100

D:\MS\1stSem\Anaconda\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1673/1673 [==============================] - 0s 64us/step - loss: 0.6709 - acc: 0.6175
Epoch 4/1000
1673/1673 [==============================] - 0s 65us/step - loss: 0.6659 - acc: 0.6204
Epoch 5/1000
1673/1673 [==============================] - 0s 65us/step - loss: 0.6701 - acc: 0.6210
Epoch 6/1000
1673/1673 [==============================] - 0s 67us/step - loss: 0.6665 - acc: 0.6186
Epoch 7/1000
1673/1673 [==============================] - 0s 56us/step - loss: 0.6666 - acc: 0.6210
Epoch 8/1000
1673/1673 [==============================] - 0s 56us/step - loss: 0.6629 - acc: 0.6192
Epoch 9/1000
1673/1673 [==============================] - 0s 65us/step - loss: 0.6631 - acc: 0.6204
Epoch 10/1000
1673/1673 [==============================] - 0s 65us/step - loss: 0.6639 - acc: 0.6216
Epoch 11/1000
1673/1673 [==============================] - 0s 63us/step - loss: 0.6614 - acc: 0.6204
Epoch 12/1000
1673/1673 [==============================] - 0s 65us/step - loss: 0.6619 - acc: 0.6210
Epoch 13/1

1673/1673 [==============================] - 0s 31us/step - loss: 0.6639 - acc: 0.6605
Epoch 85/1000
1673/1673 [==============================] - 0s 31us/step - loss: 0.5957 - acc: 0.6856
Epoch 86/1000
1673/1673 [==============================] - 0s 29us/step - loss: 0.5600 - acc: 0.7197
Epoch 87/1000
1673/1673 [==============================] - 0s 29us/step - loss: 0.5271 - acc: 0.7555
Epoch 88/1000
1673/1673 [==============================] - 0s 29us/step - loss: 0.5123 - acc: 0.7603
Epoch 89/1000
1673/1673 [==============================] - 0s 29us/step - loss: 0.5072 - acc: 0.7621
Epoch 90/1000
1673/1673 [==============================] - 0s 33us/step - loss: 0.5169 - acc: 0.7579
Epoch 91/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.5102 - acc: 0.7645
Epoch 92/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.5157 - acc: 0.7663
Epoch 93/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.4921 - acc: 0.7830
Epoc

1673/1673 [==============================] - 0s 26us/step - loss: 0.4236 - acc: 0.8183
Epoch 165/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.4282 - acc: 0.8213
Epoch 166/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.4274 - acc: 0.8165
Epoch 167/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.4241 - acc: 0.8147
Epoch 168/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.4279 - acc: 0.8296
Epoch 169/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.4289 - acc: 0.8183
Epoch 170/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.4274 - acc: 0.8189
Epoch 171/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.4273 - acc: 0.8213
Epoch 172/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.4255 - acc: 0.8249
Epoch 173/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.4227 - acc: 0.

...

1673/1673 [==============================] - 0s 26us/step - loss: 0.3792 - acc: 0.8428
Epoch 644/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3688 - acc: 0.8404
Epoch 645/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3698 - acc: 0.8482
Epoch 646/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3728 - acc: 0.8500
Epoch 647/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3732 - acc: 0.8458
Epoch 648/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3775 - acc: 0.8470
Epoch 649/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3735 - acc: 0.8476
Epoch 650/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3713 - acc: 0.8530
Epoch 651/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3702 - acc: 0.8500
Epoch 652/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3739 - acc: 0.

1673/1673 [==============================] - 0s 26us/step - loss: 0.3693 - acc: 0.8494
Epoch 724/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3649 - acc: 0.8494
Epoch 725/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3651 - acc: 0.8488
Epoch 726/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3806 - acc: 0.8446
Epoch 727/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3659 - acc: 0.8488
Epoch 728/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.3685 - acc: 0.8440
Epoch 729/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3744 - acc: 0.8464
Epoch 730/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0.3647 - acc: 0.8506
Epoch 731/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3671 - acc: 0.8500
Epoch 732/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3658 - acc: 0.

1673/1673 [==============================] - 0s 24us/step - loss: 0.3584 - acc: 0.8613
Epoch 804/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3756 - acc: 0.8458
Epoch 805/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3683 - acc: 0.8530
Epoch 806/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3618 - acc: 0.8571
Epoch 807/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3688 - acc: 0.8470
Epoch 808/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3641 - acc: 0.8548
Epoch 809/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3625 - acc: 0.8530
Epoch 810/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3618 - acc: 0.8583
Epoch 811/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3722 - acc: 0.8482
Epoch 812/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3707 - acc: 0.

Epoch 883/1000
1673/1673 [==============================] - 0s 29us/step - loss: 0.3669 - acc: 0.8500
Epoch 884/1000
1673/1673 [==============================] - 0s 29us/step - loss: 0.3618 - acc: 0.8553
Epoch 885/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3598 - acc: 0.8607
Epoch 886/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3675 - acc: 0.8488
Epoch 887/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3593 - acc: 0.8548
Epoch 888/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3623 - acc: 0.8553
Epoch 889/1000
1673/1673 [==============================] - ETA: 0s - loss: 0.3519 - acc: 0.862 - 0s 24us/step - loss: 0.3656 - acc: 0.8524
Epoch 890/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3695 - acc: 0.8494
Epoch 891/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3648 - acc: 0.8530
Epoch 892/1000
1673/1673 [==================

Epoch 963/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3616 - acc: 0.8548
Epoch 964/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3571 - acc: 0.8607
Epoch 965/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3680 - acc: 0.8512
Epoch 966/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3623 - acc: 0.8536
Epoch 967/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3625 - acc: 0.8530
Epoch 968/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3640 - acc: 0.8500
Epoch 969/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3640 - acc: 0.8536
Epoch 970/1000
1673/1673 [==============================] - 0s 24us/step - loss: 0.3601 - acc: 0.8548
Epoch 971/1000
1673/1673 [==============================] - 0s 22us/step - loss: 0.3619 - acc: 0.8500
Epoch 972/1000
1673/1673 [==============================] - 0s 26us/step - loss: 0

In [377]:
wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(TrainingSubtractData),TrainingTarget):
    y = model.predict(np.array(i).reshape(-1,9))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Training Accuracy: " + str(right/(right+wrong)*100))

wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(ValDataSubtract),ValDataAct):
    y = model.predict(np.array(i).reshape(-1,9))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Validation Accuracy: " + str(right/(right+wrong)*100))

wrong   = 0
right   = 0
predictedTestLabel = []
for i,j in zip(np.transpose(TestDataSubtract),TestDataAct):
    y = model.predict(np.array(i).reshape(-1,9))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Testing Accuracy: " + str(right/(right+wrong)*100))

Errors: 231  Correct :1442
Training Accuracy: 86.19246861924687
Errors: 32  Correct :177
Validation Accuracy: 84.688995215311
Errors: 36  Correct :172
Testing Accuracy: 82.6923076923077



## Neural Network Solution for Feature Addition

In [378]:
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.callbacks import EarlyStopping, TensorBoard
from keras.utils import np_utils
import numpy as np

TrainingTarget1 = np_utils.to_categorical(np.array(TrainingTarget),2)

In [379]:
input_size = 18
drop_out = 0.2
first_dense_layer_nodes  = 256
second_dense_layer_nodes = 2

def get_model_add():
    model = Sequential()
    model.add(Dense(first_dense_layer_nodes, input_dim=input_size))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(first_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.add(Dropout(drop_out)) 
    
    model.add(Dense(second_dense_layer_nodes))
    model.add(Activation('sigmoid')) 
    model.summary()
    
    model.compile(optimizer='Adadelta',
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    
    return model

In [380]:
model = get_model_add()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_80 (Dense)             (None, 256)               4864      
_________________________________________________________________
activation_80 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_53 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_81 (Dense)             (None, 256)               65792     
_________________________________________________________________
activation_81 (Activation)   (None, 256)               0         
_________________________________________________________________
dropout_54 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_82 (Dense)             (None, 2)                 514       
__________

In [381]:
validation_data_split = 0
num_epochs = 1000
model_batch_size = 128
tb_batch_size = 32
early_patience = 10
tensorboard_cb   = TensorBoard(log_dir='logs', batch_size= tb_batch_size, write_graph= True)
earlystopping_cb = EarlyStopping(monitor='val_loss', verbose=1, patience=early_patience, mode='min')
history = model.fit(np.transpose(TrainingAddData)
                    , TrainingTarget1
                    , validation_split=validation_data_split
                    , epochs=num_epochs
                    , batch_size=model_batch_size
                    , callbacks = [tensorboard_cb,earlystopping_cb])

Epoch 1/1000
1673/1673 [==============================] - 1s 708us/step - loss: 0.6754 - acc: 0.6133
Epoch 2/1000
 768/1673 [============>.................] - ETA: 0s - loss: 0.6613 - acc: 0.6328

D:\MS\1stSem\Anaconda\lib\site-packages\keras\callbacks.py:535: RuntimeWarning: Early stopping conditioned on metric `val_loss` which is not available. Available metrics are: acc,loss
  (self.monitor, ','.join(list(logs.keys()))), RuntimeWarning


1673/1673 [==============================] - 0s 191us/step - loss: 0.6649 - acc: 0.6210
Epoch 3/1000
1673/1673 [==============================] - 0s 198us/step - loss: 0.6588 - acc: 0.6240
Epoch 4/1000
1673/1673 [==============================] - 0s 191us/step - loss: 0.6542 - acc: 0.6252
Epoch 5/1000
1673/1673 [==============================] - 0s 218us/step - loss: 0.6456 - acc: 0.6300
Epoch 6/1000
1673/1673 [==============================] - 0s 177us/step - loss: 0.6400 - acc: 0.6348
Epoch 7/1000
1673/1673 [==============================] - 0s 172us/step - loss: 0.6340 - acc: 0.6324
Epoch 8/1000
1673/1673 [==============================] - 0s 151us/step - loss: 0.5969 - acc: 0.6766
Epoch 9/1000
1673/1673 [==============================] - 0s 122us/step - loss: 0.5856 - acc: 0.7131
Epoch 10/1000
1673/1673 [==============================] - 0s 115us/step - loss: 0.5668 - acc: 0.7095
Epoch 11/1000
1673/1673 [==============================] - 0s 112us/step - loss: 0.5571 - acc: 0.7268
E

1673/1673 [==============================] - 0s 45us/step - loss: 0.3069 - acc: 0.8924
Epoch 83/1000
1673/1673 [==============================] - 0s 45us/step - loss: 0.3130 - acc: 0.8924
Epoch 84/1000
1673/1673 [==============================] - 0s 50us/step - loss: 0.3138 - acc: 0.8918
Epoch 85/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.3117 - acc: 0.8924
Epoch 86/1000
1673/1673 [==============================] - 0s 45us/step - loss: 0.3052 - acc: 0.8858
Epoch 87/1000
1673/1673 [==============================] - 0s 41us/step - loss: 0.3041 - acc: 0.8942
Epoch 88/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.3031 - acc: 0.8918
Epoch 89/1000
1673/1673 [==============================] - 0s 41us/step - loss: 0.3050 - acc: 0.8912
Epoch 90/1000
1673/1673 [==============================] - 0s 41us/step - loss: 0.3096 - acc: 0.8948
Epoch 91/1000
1673/1673 [==============================] - 0s 45us/step - loss: 0.2988 - acc: 0.8954
Epoc

1673/1673 [==============================] - 0s 43us/step - loss: 0.2434 - acc: 0.9163
Epoch 163/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.2391 - acc: 0.9175
Epoch 164/1000
1673/1673 [==============================] - 0s 41us/step - loss: 0.2355 - acc: 0.9193
Epoch 165/1000
1673/1673 [==============================] - 0s 38us/step - loss: 0.2324 - acc: 0.9223
Epoch 166/1000
1673/1673 [==============================] - 0s 41us/step - loss: 0.2366 - acc: 0.9205
Epoch 167/1000
1673/1673 [==============================] - 0s 40us/step - loss: 0.2334 - acc: 0.9235
Epoch 168/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.2359 - acc: 0.9205
Epoch 169/1000
1673/1673 [==============================] - 0s 38us/step - loss: 0.2255 - acc: 0.9295
Epoch 170/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.2280 - acc: 0.9235
Epoch 171/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.2257 - acc: 0.

...

1673/1673 [==============================] - 0s 37us/step - loss: 0.0745 - acc: 0.9767
Epoch 643/1000
1673/1673 [==============================] - 0s 28us/step - loss: 0.0656 - acc: 0.9803
Epoch 644/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0596 - acc: 0.9809
Epoch 645/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0684 - acc: 0.9803
Epoch 646/1000
1673/1673 [==============================] - 0s 40us/step - loss: 0.0635 - acc: 0.9797
Epoch 647/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0638 - acc: 0.9821
Epoch 648/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.0647 - acc: 0.9803
Epoch 649/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0663 - acc: 0.9815
Epoch 650/1000
1673/1673 [==============================] - 0s 47us/step - loss: 0.0683 - acc: 0.9785
Epoch 651/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0591 - acc: 0.

1673/1673 [==============================] - 0s 36us/step - loss: 0.0482 - acc: 0.9857
Epoch 723/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0568 - acc: 0.9839
Epoch 724/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0579 - acc: 0.9839
Epoch 725/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0520 - acc: 0.9857
Epoch 726/1000
1673/1673 [==============================] - 0s 38us/step - loss: 0.0518 - acc: 0.9833
Epoch 727/1000
1673/1673 [==============================] - 0s 33us/step - loss: 0.0558 - acc: 0.9827
Epoch 728/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0519 - acc: 0.9839
Epoch 729/1000
1673/1673 [==============================] - 0s 40us/step - loss: 0.0491 - acc: 0.9851
Epoch 730/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0521 - acc: 0.9857
Epoch 731/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0540 - acc: 0.

1673/1673 [==============================] - 0s 42us/step - loss: 0.0474 - acc: 0.9845
Epoch 803/1000
1673/1673 [==============================] - 0s 45us/step - loss: 0.0469 - acc: 0.9845
Epoch 804/1000
1673/1673 [==============================] - 0s 53us/step - loss: 0.0422 - acc: 0.9863
Epoch 805/1000
1673/1673 [==============================] - 0s 50us/step - loss: 0.0484 - acc: 0.9863
Epoch 806/1000
1673/1673 [==============================] - 0s 50us/step - loss: 0.0491 - acc: 0.9827
Epoch 807/1000
1673/1673 [==============================] - 0s 55us/step - loss: 0.0468 - acc: 0.9845
Epoch 808/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.0544 - acc: 0.9845
Epoch 809/1000
1673/1673 [==============================] - 0s 41us/step - loss: 0.0559 - acc: 0.9821
Epoch 810/1000
1673/1673 [==============================] - 0s 41us/step - loss: 0.0461 - acc: 0.9851
Epoch 811/1000
1673/1673 [==============================] - 0s 38us/step - loss: 0.0511 - acc: 0.

1673/1673 [==============================] - 0s 37us/step - loss: 0.0422 - acc: 0.9845
Epoch 883/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0375 - acc: 0.9886
Epoch 884/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0451 - acc: 0.9833
Epoch 885/1000
1673/1673 [==============================] - 0s 40us/step - loss: 0.0456 - acc: 0.9868
Epoch 886/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0469 - acc: 0.9880
Epoch 887/1000
1673/1673 [==============================] - 0s 28us/step - loss: 0.0517 - acc: 0.9857
Epoch 888/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0387 - acc: 0.9857
Epoch 889/1000
1673/1673 [==============================] - 0s 28us/step - loss: 0.0496 - acc: 0.9851
Epoch 890/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0435 - acc: 0.9857
Epoch 891/1000
1673/1673 [==============================] - 0s 37us/step - loss: 0.0400 - acc: 0.

1673/1673 [==============================] - 0s 48us/step - loss: 0.0434 - acc: 0.9886
Epoch 963/1000
1673/1673 [==============================] - 0s 45us/step - loss: 0.0399 - acc: 0.9874
Epoch 964/1000
1673/1673 [==============================] - 0s 48us/step - loss: 0.0423 - acc: 0.9868
Epoch 965/1000
1673/1673 [==============================] - 0s 45us/step - loss: 0.0372 - acc: 0.9886
Epoch 966/1000
1673/1673 [==============================] - 0s 55us/step - loss: 0.0446 - acc: 0.9863
Epoch 967/1000
1673/1673 [==============================] - 0s 45us/step - loss: 0.0412 - acc: 0.9868
Epoch 968/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.0335 - acc: 0.9886
Epoch 969/1000
1673/1673 [==============================] - 0s 50us/step - loss: 0.0389 - acc: 0.9874
Epoch 970/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.0403 - acc: 0.9880
Epoch 971/1000
1673/1673 [==============================] - 0s 43us/step - loss: 0.0362 - acc: 0.

In [382]:
wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(TrainingAddData),TrainingTarget):
    y = model.predict(np.array(i).reshape(-1,18))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Training Accuracy: " + str(right/(right+wrong)*100))

wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(ValDataAdd),ValDataAct):
    y = model.predict(np.array(i).reshape(-1,18))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Validation Accuracy: " + str(right/(right+wrong)*100))

wrong   = 0
right   = 0
predictedTestLabel = []

for i,j in zip(np.transpose(TestDataAdd),TestDataAct):
    y = model.predict(np.array(i).reshape(-1,18))
    predictedTestLabel.append(y.argmax())
    
    if j == y.argmax():
        right = right + 1
    else:
        wrong = wrong + 1

print("Errors: " + str(wrong), " Correct :" + str(right))
print("Testing Accuracy: " + str(right/(right+wrong)*100))

Errors: 17  Correct :1656
Training Accuracy: 98.98386132695755
Errors: 4  Correct :205
Validation Accuracy: 98.08612440191388
Errors: 4  Correct :204
Testing Accuracy: 98.07692307692307
